# Análisis y visualización de datos
### Actividad 2
### Situación
- Una vez que se entregó la primera parte del informe, continuás con la segunda parte, enfocada en los aspectos demográficos y sociales. Los datos para esta sección no están completamente listos para analizar, necesitan preprocesamiento antes de poder construir los gráficos o tablas. Como sabés, algunas de estas tareas son más prácticas y eficientes haciéndolas en Python que manualmente (en Excel, por ejemplo).
### Requerimientos
- Para realizar esta actividad descargá el archivo ZIP, que encontrarás en la plataforma al final de este documento.
Importante: AI momento de importar los archivos .csv, usando pd.read_csv(), usar la opción encoding = "latin-1 " para que importe correctamente los nombres de las provincias con tildes.
### Consignas
- Como próximo paso, decidís integrar distintas bases de datos que tenés disponibles. Estas contienen variables demográficas: población, hogares y viviendas, esperanza de vida y fecundidad.
1. En Jupyter, creá un nuevo notebook e importó las librerías necesarias, y luego la base de proyecciones de población por año (en formato .csv) y las otras (ej. expectativa de vida, fecundidad). Tené en cuenta que algunas bases contienen datos de varios años y otras tienen únicamente el año del censo 2010. Hacé los chequeos básicos (head, describe, etc.).
2. Calculá un campo nuevo, densidad (población/superficie) y usar la función descrita sobre ese campo nuevo.
3. Identificá si existe algún valor extremo en la densidad de población y explicó a qué podría deberse esto.

In [1]:
a = 10
print(a)

10
